In [3]:
import pandas as pd
import networkx as nx
from math import exp
import matplotlib.pyplot as plt
import numpy as np

nodes = pd.read_csv('routes.csv', header=None, names=['i', 'j'])
g = nx.from_pandas_edgelist(nodes, 'i', 'j')
print('Number of nodes', g.number_of_nodes())
print('Number of edges', g.number_of_edges())

Number of nodes 151
Number of edges 1851


In [4]:
# We have a dataframe of countries (nodes)
countries = pd.read_csv('data.csv')
countries.set_index('country', inplace=True)
countries['population'] = 1000000*countries['population']
# Travellers are for an entire year, but our timeline is weekly
countries['travellers'] = countries['travellers']/52

countries.head()

,population,travellers,hdi
country,,,
Afghanistan,32760000.0,37113.596154,0.498
Albania,2900000.0,79730.769231,0.773
Algeria,39100000.0,54596.153846,0.747
Angola,26920000.0,23932.519231,0.581
Antigua and Barbuda,98000.0,19996.326923,0.780


In [5]:
# For each country, we need the travellers going from the neighboring countries
def return_list_neighbors(i, g):
    return [j for j in g.neighbors(i)]
print('Albania\'s neighbors :', return_list_neighbors('Albania', g))

# For all countries
def compute_sum_travelers_neighboring_countries(g):
    sum_travel_neighbors = []
    for i in countries.index:
        neighbors = return_list_neighbors(i, g)
        sum = 0
        for n in neighbors:
            sum += countries.loc[n]['travellers']
        sum_travel_neighbors.append(sum)
    return sum_travel_neighbors
countries['sum_travel_neighbors'] = compute_sum_travelers_neighboring_countries(g)

countries.head()

Albania's neighbors : ['Austria', 'Germany', 'Greece', 'Italy', 'Slovenia', 'Turkey', 'United Kingdom']


,population,travellers,hdi,sum_travel_neighbors
country,,,,
Afghanistan,32760000.0,37113.596154,0.498,2.739243e+06
Albania,2900000.0,79730.769231,0.773,3.826346e+06
Algeria,39100000.0,54596.153846,0.747,1.192336e+07
Angola,26920000.0,23932.519231,0.581,9.622885e+06
Antigua and Barbuda,98000.0,19996.326923,0.780,6.018484e+06


In [25]:
# W_ij the weight between country i and j represents the number of travellers between i and j
def compute_edge_weight(i, j):
    return countries.loc[i]['travellers']*countries.loc[j]['travellers']/(countries.loc[i]['sum_travel_neighbors'])
        
def add_edge_weights():
    for e in g.edges():
        g[e[0]][e[1]]['weight'] = compute_edge_weight(e[0], e[1])

add_edge_weights()
print('France - Germany :', g['France']['Germany'])
print('France - UK :', g['France']['United Kingdom'])

France - Germany : {'weight': 88582.44661152952}
France - UK : {'weight': 62802.53397113779}


In [37]:
# Initialisation for our model :
r0 = 3
alpha = 0.06
beta = 1
period = 20
mortality_rate = 0.2
country_start = 'France'
nb_infected_start = 100

# # Initialisation for t=0 :
countries['incubation0'] = np.zeros(countries.shape[0])
countries['sick0'] = np.zeros(countries.shape[0])
countries['recovered0'] = np.zeros(countries.shape[0])
countries['infected0'] = np.zeros(countries.shape[0])
countries['healthy0'] = countries['population']
# We infect the starting country with a certain number of people
countries.loc[country_start]['infected0'] = nb_infected_start
countries.loc[country_start]['incubation0'] = nb_infected_start
countries.loc[country_start]['healthy0'] = countries.loc[country_start]['population'] - countries.loc[country_start]['incubation0']

In [10]:
# Contamination decreases with time
def time(t):
    return exp(-alpha*t)

# We calculate the average number infected by an incubated person in a week
# The higher the HDI of a country, the less people will be infected
def reproduction_number_by_country(t):
    return (1 -beta*countries['hdi'] + 0.75)*r0*time(t)

ri = reproduction_number_by_country(1)
print('Afghanistan :', ri['Afghanistan'])
print('France :', ri['France'])

Afghanistan : 3.5372675881424382
France : 2.418451322244351


In [30]:
'''def number_incubated(t):
    ri = reproduction_number_by_country(t)
    incubation_staying = {}
    incubation_coming = {}
    for i in countries.index:
        incubation_staying[i] = countries.loc[i]['incubation'+str(t-1)]*(1 - 
                                countries.loc[i]['travellers']/(countries.loc[i]['healthy'+str(t-1)]
                        - countries.loc[i]['sick'+str(t-1)]))
        #incubation coming
        neighbors = return_list_neighbors(i, g)
        sum = 0
        for j in neighbors:
            sum += g[j][i]['weight']*countries.loc[j]['incubation'+str(t-1)]/(countries.loc[j]['healthy'+str(t-1)] -
                                      countries.loc[j]['sick'+str(t-1)])
        incubation_coming[i] = sum
    
    countries['recovered'+str(t)] = (1 - mortality_rate)*countries['sick'+str(t-1)]
    countries['infected'+str(t)] = [((incubation_staying[i] 
                + incubation_coming[i])*(1 + ri[i]) - 
                countries.loc[i]['recovered'+str(t)]) for i in countries.index]
    
    countries['healthy'+str(t)] = [countries.loc[i]['population'] - countries.loc[i]['infected'+str(t)] for i in countries.index]
    countries['sick'+str(t)] = countries['incubation'+str(t - 1)] 
    countries['incubation'+str(t)] = countries['infected'+str(t)] - countries['sick'+str(t)]'''

# Incubated people staying in the country at time t
def number_incubated_staying(t):
    incubated_staying = {}
    for i in countries.index:
        incubated_staying[i] = countries.loc[i]['incubation'+str(t-1)]*(1 - 
                    countries.loc[i]['travellers']/(countries.loc[i]['healthy'+str(t-1)] - 
                    countries.loc[i]['sick'+str(t-1)]))
    return incubated_staying
# Incubated people coming to the country at time t
def number_incubated_coming(t):
    incubated_coming = {}
    for i in countries.index:
        neighbors = return_list_neighbors(i, g)
        sum = 0
        for j in neighbors:
            sum += g[j][i]['weight']*countries.loc[j]['incubation'+str(t-1)]/(countries.loc[j]['healthy'+str(t-1)] - 
                                                                              countries.loc[j]['sick'+str(t-1)])
        incubated_coming[i] = sum
    return incubated_coming

incubated_coming = number_incubated_coming(1)
print('Number of incubated staying in France :', number_incubated_staying(1)['France'])
print('Number of incubated coming to France :', incubated_coming['France'])
print('Number of incubated coming to Germany :', incubated_coming['Germany'])
print('Number of incubated coming to the UK :', incubated_coming['United Kingdom'])

sum = 0
for n in return_list_neighbors('France', g):
    sum += incubated_coming[n]
print('For all of France\' neighbors :', sum)

Number of incubated staying in France : 98.05177317553934
Number of incubated coming to France : 0.0
Number of incubated coming to Germany : 0.1379791037237278
Number of incubated coming to the UK : 0.09782341401020529
For all of France' neighbors : 2.0675078328777334


In [38]:
def number_infected(t):
    ri = reproduction_number_by_country(t)
    incubated_staying = number_incubated_staying(t)
    incubated_coming = number_incubated_coming(t)
    
    countries['recovered'+str(t)] = (1 - mortality_rate)*countries['sick'+str(t-1)]
    countries['infected'+str(t)] = [((incubated_staying[i] 
                + incubated_coming[i])*(1 + ri[i]) - 
                countries.loc[i]['recovered'+str(t)]) for i in countries.index]
    
    countries['healthy'+str(t)] = [countries.loc[i]['population'] - countries.loc[i]['infected'+str(t)] for i in countries.index]
    countries['sick'+str(t)] = countries['incubation'+str(t - 1)] 
    countries['incubation'+str(t)] = countries['infected'+str(t)] - countries['sick'+str(t)]

for p in range(1, period+1):
    number_infected(p)
    print(p)
    print('France :', countries.loc['France']['infected'+str(p)],  countries.loc['France']['healthy'+str(p)])
    print('Germany :', countries.loc['Germany']['infected'+str(p)],  countries.loc['Germany']['healthy'+str(p)])
    print('UK :', countries.loc['United Kingdom']['infected'+str(p)],  countries.loc['United Kingdom']['healthy'+str(p)])
    print('Afghanistan :', countries.loc['Afghanistan']['infected'+str(p)],  countries.loc['Afghanistan']['healthy'+str(p)])

1
France : 335.18521366032564 64199664.81478634
Germany : 0.4576409163300908 81499999.54235908
UK : 0.32749508232947233 64999999.67250492
Afghanistan : 0.0 32759999.999999996
2
France : 675.8654048817559 64199324.13459512
Germany : 2.5035774960801844 81499997.4964225
UK : 1.8038196385521936 64999998.19618036
Afghanistan : 0.005324235079212243 32759999.994675763
3
France : 1170.9331134400036 64198829.06688656
Germany : 7.79785146294418 81499992.20214854
UK : 5.668735355577274 64999994.331264645
Afghanistan : 0.04271575587129194 32759999.957284242
4
France : 1810.2981525595046 64198189.70184744
Germany : 18.335644457517464 81499981.66435555
UK : 13.469355712937032 64999986.53064429
Afghanistan : 0.19311215118454067 32759999.806887846
5
France : 2490.2834736459413 64197509.71652635
Germany : 35.36126428893991 81499964.63873571
UK : 26.283707086185657 64999973.71629291
Afghanistan : 0.6498268905386356 32759999.350173105
6
France : 2997.504613871854 64197002.49538613
Germany : 57.4559982604